In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
import json
import random
from fp.fp import FreeProxy
from fake_useragent import UserAgent

In [3]:
def getIP(rand=True, anonym=True):
    ua = UserAgent()
    proxy = FreeProxy( rand=rand, anonym=anonym).get()
    ip = proxy.split("://")[1]
    return ua.random, ip

def initDriver(rotated_proxy=False):
    # setting webdriver selenium -- aviod bot detected
    option = webdriver.ChromeOptions()
    # Removes navigator.webdriver flagIngest
    option.add_experimental_option("excludeSwitches", ["enable-automation"])
    option.add_experimental_option('useAutomationExtension', False)

    option.add_argument('--disable-blink-features=AutomationControlled')
    option.add_argument("window-size=1280,800")
    
    if rotated_proxy == True: # if false = have prox
        print('rotated proxy')
        # NOTE: comment when not want to use
        userAgent, proxyIP = getIP()
        print('proxy =', proxyIP)
        option.add_argument(f'proxy-server={proxyIP}')
        option.add_argument(f'user-agent={userAgent}')
        
    return webdriver.Chrome(executable_path='chromedriver.exe', options=option)

In [3]:
def get_hotel_list():
  url = 'https://www.tripadvisor.com/Hotels-g28932-Hawaii-Hotels.html'
  driver = initDriver(rotated_proxy=False)
  driver.get(url) # open driver to 
  # click see all
  see_all_button = driver.find_element("xpath", '/html/body/div[2]/div[1]/div[2]/div/div[2]/div[3]/div[2]/div[9]/div/div/button')  # new version
  see_all_button.click()

  import bs4
  # change markup lang string html to object{} 
  data = driver.page_source
  soup = bs4.BeautifulSoup(data,"html.parser")
  hotels_list_tag = soup.find("div", {"id": 'map_wc_dusty_bridge'}).find_all('div')[1]
  hotels_list_30 = hotels_list_tag.attrs['data-hotels-data']
  hotels_list_30 = json.loads(hotels_list_30)
  hotels_list_30['geoName'] = 'Hawaii'


  return hotels_list_30

In [4]:
# --- call function to get list top 30 hotels of Hawaii ----
# hotels_list_30 =  get_hotel_list()

# # save file json
  # with open('./data/Hawaii_hotel_list_30.json', 'w') as fp:
  #   json.dump(hotels_list_30, fp)

In [5]:
# # load data from json
# with open('./data/Hawaii_hotel_list_30.json', 'r') as fp:
#   hotels_list_30 = json.load(fp)
  
# hotels_list_30 = hotels_list_30['hotels']

----------------

In [13]:
import sys
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException

def get_reviews(url, num_page=3):
    # import the webdriver
    driver = initDriver()
    driver.get(url)

    # change the value inside the range to save more or less reviews
    review_list = []
    page = 0
    for i in range(0, num_page):
        page +=1 
        print(f'page [{page}/{num_page}]')           
        
        time.sleep(1)
        # driver.find_element(By.XPATH,".//div[contains(@data-test-target, 'expand-review')]").click()
        # expand the review
        try:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]"))).click()
        except StaleElementReferenceException:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]"))).click()
        except ElementClickInterceptedException:
            driver.refresh()
            time.sleep(3)
            WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]"))).click()
        except TimeoutException:
            driver.refresh()
            time.sleep(3)
            WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]"))).click()
             
        except Exception:
            try:
                driver.find_element(By.XPATH,'.//a[@class="ui_button nav next primary "]').click()
            except NoSuchElementException:
                break
            
        
        # while True:
        #     element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]")))
        #     if element != None:
        #         element.click()
        #         break
        #     else:
        #         driver.refresh()
        #         time.sleep(2)
        #         continue 
    
        
        container = driver.find_elements(By.XPATH,"//div[@data-reviewid]")
        dates = driver.find_elements(By.XPATH,".//div[@class='sCZGP']")

        for j in range(len(container)):
            data_reviewid = container[j].get_attribute('data-reviewid') 
            # print('data_reviewid:',data_reviewid)
            rating = container[j].find_element(By.XPATH,".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
            title = container[j].find_element(By.XPATH,".//div[@class='KgQgP MC _S b S6 H5 _a']").text
            review = container[j].find_element(By.XPATH,".//div[@class='fIrGe _T']").text.replace('\n','  ')
            review_date = dates[j].text.split('wrote a review')[1].split('\n')[0].strip()
            
            try:
                origin = dates[j].find_element(By.XPATH,".//span[contains(@class, 'default LXUOn small')]").text
            except NoSuchElementException:
                origin = None
            try:
                stay_date = container[j].find_element(By.XPATH,".//span[contains(@class, 'teHYY _R Me S4 H3')]").text.split(':')[1].strip()
            except NoSuchElementException:
                stay_date = None
            try:
                trip_type = container[j].find_element(By.XPATH,".//span[contains(@class, 'TDKzw _R Me')]").text.split(':')[1].strip()
                # trip_type = container[j].find_element(By.XPATH,".//span[contains(@class, 'TDKzw _R Me')]").text
            except NoSuchElementException:
                trip_type = None
            try:
                room_tip = container[j].find_element(By.XPATH,".//div[@class='Pb']").text.split(':')[1].strip()
            except NoSuchElementException:
                room_tip = None
                
            review_list.append([data_reviewid,rating,title,review,review_date,stay_date,trip_type,room_tip,origin])

        # change the page, note: not found in last page
        try:
            driver.find_element(By.XPATH,'.//a[@class="ui_button nav next primary "]').click()
        except NoSuchElementException:
            break

    driver.quit()
    columnName = ['reviewid','rating','title','review','review_date','stay_date','trip_type','room_tip','origin']
    return pd.DataFrame(data=review_list, columns=columnName) # df

In [11]:
# loop all hotel_list and thier numReviews
with open('./data/Hawaii_hotel_list_30.json', 'r') as fp:
  hotels_list_30 = json.load(fp)

base_url = 'https://www.tripadvisor.com'  

In [16]:
# calling for test (run one url)
# 0,7,
idx = 1 # see in _hotel_list.json

hotel_name = hotels_list_30["geoName"]
url = base_url + hotels_list_30["hotels"][idx]['detailUrl']
num_page = round(hotels_list_30["hotels"][idx]['numReviews'] / 10)
hotel_id = hotels_list_30["hotels"][idx]["id"]
df = get_reviews(url,num_page)
df.to_csv(f'./data/{hotel_name}_{hotel_id}.csv',  index=False)

<ipython-input-3-80ef26bf8993>:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(executable_path='chromedriver.exe', options=option)


page [1/262]
page [2/262]
page [3/262]
page [4/262]
page [5/262]
page [6/262]
page [7/262]
page [8/262]
page [9/262]
page [10/262]
page [11/262]
page [12/262]
page [13/262]
page [14/262]
page [15/262]
page [16/262]
page [17/262]
page [18/262]
page [19/262]
page [20/262]
page [21/262]
page [22/262]
page [23/262]
page [24/262]
page [25/262]
page [26/262]
page [27/262]
page [28/262]
page [29/262]
page [30/262]
page [31/262]
page [32/262]
page [33/262]
page [34/262]
page [35/262]
page [36/262]
page [37/262]
page [38/262]
page [39/262]
page [40/262]
page [41/262]
page [42/262]
page [43/262]
page [44/262]
page [45/262]
page [46/262]
page [47/262]
page [48/262]
page [49/262]
page [50/262]
page [51/262]
page [52/262]
page [53/262]
page [54/262]
page [55/262]
page [56/262]
page [57/262]
page [58/262]
page [59/262]
page [60/262]
page [61/262]
page [62/262]
page [63/262]
page [64/262]
page [65/262]
page [66/262]
page [67/262]
page [68/262]
page [69/262]
page [70/262]
page [71/262]
page [72/262]
p

In [27]:
len(hotels_list_30['hotels'][2:7])

5

In [28]:
# loop all hotel_list and thier numReviews
with open('./data/Hawaii_hotel_list_30.json', 'r') as fp:
  hotels_list_30 = json.load(fp)

base_url = 'https://www.tripadvisor.com'  

queue_scrape = hotels_list_30['hotels'][2:7]
for hotel in queue_scrape:
  # print(hotel['numReviews'])
  url = base_url + hotel['detailUrl']
  total_page = round(hotel['numReviews'] / 10)
  hotel_id = hotel["id"]
  print(f'...scraping url:{url}')
  df = get_reviews(url, total_page)
  df.to_csv(f'./data/{hotels_list_30["geoName"]}_{hotel["id"]}.csv',  index=False)
  # reviews = pd.concat([reviews, df], axis=0) # mock total_page: not send this param

...scraping url:https://www.tripadvisor.com/Hotel_Review-g60634-d101357-Reviews-Ka_anapali_Beach_Hotel-Lahaina_Maui_Hawaii.html


<ipython-input-3-80ef26bf8993>:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(executable_path='chromedriver.exe', options=option)


page [1/827]
page [2/827]
page [3/827]
page [4/827]
page [5/827]
page [6/827]
page [7/827]
page [8/827]
page [9/827]
page [10/827]
page [11/827]
page [12/827]
page [13/827]
page [14/827]
page [15/827]
page [16/827]
page [17/827]
page [18/827]
page [19/827]
page [20/827]
page [21/827]
page [22/827]
page [23/827]
page [24/827]
page [25/827]
page [26/827]
page [27/827]
page [28/827]
page [29/827]
page [30/827]
page [31/827]
page [32/827]
page [33/827]
page [34/827]
page [35/827]
page [36/827]
page [37/827]
page [38/827]
page [39/827]
page [40/827]
page [41/827]
page [42/827]
page [43/827]
page [44/827]
page [45/827]
page [46/827]
page [47/827]
page [48/827]
page [49/827]
page [50/827]
page [51/827]
page [52/827]
page [53/827]
page [54/827]
page [55/827]
page [56/827]
page [57/827]
page [58/827]
page [59/827]
page [60/827]
page [61/827]
page [62/827]
page [63/827]
page [64/827]
page [65/827]
page [66/827]
page [67/827]
page [68/827]
page [69/827]
page [70/827]
page [71/827]
page [72/827]
p

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00C578B3+2193587]
	Ordinal0 [0x00BF0681+1771137]
	Ordinal0 [0x00B041A8+803240]
	Ordinal0 [0x00B324A0+992416]
	Ordinal0 [0x00B3273B+993083]
	Ordinal0 [0x00B5F7C2+1177538]
	Ordinal0 [0x00B4D7F4+1103860]
	Ordinal0 [0x00B5DAE2+1170146]
	Ordinal0 [0x00B4D5C6+1103302]
	Ordinal0 [0x00B277E0+948192]
	Ordinal0 [0x00B286E6+952038]
	GetHandleVerifier [0x00F00CB2+2738370]
	GetHandleVerifier [0x00EF21B8+2678216]
	GetHandleVerifier [0x00CE17AA+512954]
	GetHandleVerifier [0x00CE0856+509030]
	Ordinal0 [0x00BF743B+1799227]
	Ordinal0 [0x00BFBB68+1817448]
	Ordinal0 [0x00BFBC55+1817685]
	Ordinal0 [0x00C05230+1856048]
	BaseThreadInitThunk [0x75E87BA9+25]
	RtlInitializeExceptionChain [0x772EBB3B+107]
	RtlClearBits [0x772EBABF+191]


-------------

## Backup code

In [83]:

# import sys
# import csv
# import time
# from selenium import webdriver
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait 
# from selenium.webdriver.support import expected_conditions as EC
# # default path to file to store data
# path_to_file = "/Users/gius/Desktop/reviews.csv"

# # default number of scraped pages
# num_page = 1

# url = 'https://www.tripadvisor.com/Hotel_Review-g60982-d87016-Reviews-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html'

# # import the webdriver
# driver = initDriver()
# driver.get(url)
# driver.find_element(By.XPATH,".//div[contains(@data-test-target, 'expand-review')]").click()

# review_list = []
# container = driver.find_elements(By.XPATH,"//div[@data-reviewid]")
# dates = driver.find_elements(By.XPATH,".//div[@class='sCZGP']")
# # print('container',container)
# print('id',container[0].get_attribute('data-reviewid'))
# for j in range(len(container)):
#     data_reviewid = container[j].get_attribute('data-reviewid') 
#     rating = container[j].find_element(By.XPATH,".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
#     title = container[j].find_element(By.XPATH,".//div[@class='KgQgP MC _S b S6 H5 _a']").text
#     review = container[j].find_element(By.XPATH,".//div[@class='fIrGe _T']").text
#     reviewDate = dates[j].text.split('wrote a review')[1].split('\n')[0].strip()
    
#     try:
#         origin = dates[j].find_element(By.XPATH,".//span[contains(@class, 'default LXUOn small')]").text
#     except NoSuchElementException:
#         origin = None
#     try:
#         stayDate = container[j].find_element(By.XPATH,".//span[contains(@class, 'teHYY _R Me S4 H3')]").text.split(':')[1].strip()
#     except NoSuchElementException:
#         stayDate = None
#     try:
#         tripType = container[j].find_element(By.XPATH,".//span[contains(@class, 'TDKzw _R Me')]").text.split(':')[1].strip()
#     except NoSuchElementException:
#         tripType = None
    
#     review_list.append([data_reviewid,rating,title,review,reviewDate,stayDate,tripType,origin])
    
# #     # print('title:',title)
# #     # print('review_date:',review_date)
# #     # print('stay_date:',stay_date)
# #     # print('origin:',origin)

# #     # df['rating'] = rating
# #     # df['title'] = title
# #     # df['review'] = review
# #     # df['review_date'] = reviewDate
# #     # df['stay_date'] = stayDate
# #     # df['trip_type'] = tripType


# # # # # change the page            
# # # # driver.find_element(By.XPATH,'.//a[@class="ui_button nav next primary "]').click()
# # # # driver.quit()
# columnName = ['reviewid','rating','title','review','review_date','stay_date','trip_type','origin']
# df = pd.DataFrame(data=review_list, columns=columnName)
# df


<ipython-input-69-80ef26bf8993>:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(executable_path='chromedriver.exe', options=option)


id 862596585


,reviewid,rating,title,review,review_date,stay_date,trip_type,origin
0,862596585,20,It is pretty unimpressed so far,I read other reviews and didn’t believe. So fa...,Sep 2022,September 2022,Traveled with family,"West Orange, New Jersey"
1,808709695,50,Excellent stay at the village,We stayed one night in the rainbow tower full ...,Sep 2021,August 2021,Traveled with family,"Queens Village, New York"
2,862587824,40,Worth it if you do your homework,Fourth time in Waikiki but first time at the H...,Sep 2022,September 2022,None,"Glasgow, United Kingdom"
3,862547086,20,Never again. Outdated and overpriced.,We stayed in Sept 2022 for 6 nights. . After a...,Sep 2022,September 2022,None,None
4,862393047,30,Overpriced and outdated,Over priced & outdated. Was placed in one of t...,Sep 2022,September 2022,None,None
5,862351021,30,Not as expected.,LOTS of people...more than I expected. Our sal...,Sep 2022,September 2022,Traveled with family,"Crescent City, California"
6,862126697,10,Stay away,Sold the prepaid room to someone else. Put us ...,Sep 2022,September 2022,Traveled with family,None
7,861949142,30,Disappointed,Hotel was a total disappointment. We were urge...,Sep 2022,September 2022,Traveled with friends,None
8,861885025,20,disappointed dinner,place was beautiful so waa view. Service was l...,Sep 2022,September 2022,Traveled as a couple,"Brewster, New York"
9,861562040,40,Aloha Spirit,"Hilton Hawaiian Village is a beautiful hotel, ...",Sep 2022,September 2022,None,None
